# Teste do projeto spot
    - pegar o token
    - extrair os dados da api
    - criar funcoes para extrair os dados 

In [100]:
import requests
import json
import geopandas
import cartoframes
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util

In [101]:
load_dotenv()

True

In [102]:
token = getpass.getpass("token: ")


In [103]:
def extrator(where, lat, log, lim):
    url = f"https://api.foursquare.com/v3/places/search?query={where}&ll={lat}%2C{log}&sort=DISTANCE&limit={lim}"
            
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    response = requests.get(url, headers=headers).json()

    return response

In [104]:
test_result = extrator('Coffee Shop', '41.3992766','2.1890409', '10')

In [105]:
test_result['results']

[{'fsq_id': '5b8521d489e490002cefef18',
  'categories': [{'id': 13035,
    'name': 'Coffee Shop',
    'short_name': 'Coffee Shop',
    'plural_name': 'Coffee Shops',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'}},
   {'id': 13065,
    'name': 'Restaurant',
    'short_name': 'Restaurant',
    'plural_name': 'Restaurants',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'}}],
  'chains': [],
  'closed_bucket': 'VeryLikelyOpen',
  'distance': 515,
  'geocodes': {'main': {'latitude': 41.397168, 'longitude': 2.183123},
   'roof': {'latitude': 41.397168, 'longitude': 2.183123}},
  'link': '/v3/places/5b8521d489e490002cefef18',
  'location': {'address': "Calle D'Ausiàs Marc, 151",
   'admin_region': 'Cataluña',
   'country': 'ES',
   'formatted_address': "Calle D'Ausiàs Marc, 151, 08013 Barcelona Catalunya",
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'}

In [106]:
test_result = extrator('Coffee Shop', '41.3992766','2.1890409', '5')

In [107]:
test_result = extrator('starbucks', '41.3992766','2.1890409', '3')

In [108]:
test_result

{'results': [{'fsq_id': '4c9b0a2cd4b1b1f7ad69d535',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
     'name': 'Starbucks'}],
   'closed_bucket': 'Unsure',
   'distance': 676,
   'geocodes': {'main': {'latitude': 41.404985, 'longitude': 2.192439},
    'roof': {'latitude': 41.404985, 'longitude': 2.192439}},
   'link': '/v3/places/4c9b0a2cd4b1b1f7ad69d535',
   'location': {'address': 'Avinguda Diagonal, 208',
    'address_extended': 'Westfield Glòries',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': 'Westfield Glòries',
    'formatted_address': 'Avinguda Diagonal, 208 (Westfield Glòries), 08018 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08018',
    'region': 'Catalunya'},
   'name': 'St

# Como enviar o json extraido da api para meu mongo

In [119]:
def enviar_para_mongodb(data):
    # Conectar ao MongoDB (certifique-se de ter o servidor MongoDB em execução)
    client = MongoClient("localhost", 27017)

    # Escolher ou criar um banco de dados e uma coleção no MongoDB
    db = client["Project_Spot"]
    collection = db["Colection"]

    # Inserir os dados no MongoDB
    result = collection.insert_one(data)

    # Imprimir o ID do documento inserido
    print(f"Documento inserido com ID: {result.inserted_id}")

    return result

In [120]:
# Extrair dados da API Foursquare
test_result = extrator('Coffee Shop', '41.3992766', '2.1890409', '10')

In [121]:
# Enviar dados extraídos para o MongoDB
enviar_para_mongodb(test_result)

Documento inserido com ID: 65c4f89e23d7612a93f0cdec


InsertOneResult(ObjectId('65c4f89e23d7612a93f0cdec'), acknowledged=True)

In [ ]:
{'results': [{'fsq_id': '4c9b0a2cd4b1b1f7ad69d535',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],

In [123]:
test_result = extrator('starbucks', '41.3992766','2.1890409', '3')
test_result['results'][0]['']

{'fsq_id': '4c9b0a2cd4b1b1f7ad69d535',
 'categories': [{'id': 13035,
   'name': 'Coffee Shop',
   'short_name': 'Coffee Shop',
   'plural_name': 'Coffee Shops',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}}],
 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
   'name': 'Starbucks'}],
 'closed_bucket': 'Unsure',
 'distance': 676,
 'geocodes': {'main': {'latitude': 41.404985, 'longitude': 2.192439},
  'roof': {'latitude': 41.404985, 'longitude': 2.192439}},
 'link': '/v3/places/4c9b0a2cd4b1b1f7ad69d535',
 'location': {'address': 'Avinguda Diagonal, 208',
  'address_extended': 'Westfield Glòries',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': 'Westfield Glòries',
  'formatted_address': 'Avinguda Diagonal, 208 (Westfield Glòries), 08018 Barcelona Catalunya',
  'locality': 'Barcelona',
  'postcode': '08018',
  'region': 'Catalunya'},
 'name': 'Starbucks',
 'related_places': {'parent': {'fsq_id': '4cb0831

In [124]:
client = MongoClient("localhost", 27017)

In [125]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)